# langChain全面剖析之Memory（下）

## 3 支持实体识别的自定义Memory

### 3.2 SpacyEntityMemory实现

上一小节安装了spacy以用于实体识别，本节实现具体的功能

#### (1) 试用Spacy

首先导入依赖实例化模型，传入一句话进行测试

In [3]:
# 导入依赖包
import spacy

# 实例化模型。
nlp = spacy.load("zh_core_web_sm")

# 测试
text = "2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。"
doc = nlp(text) # 执行
doc.ents        # 查看实体识别结果

(2014年9月, 马云, 杭州, 中国)

再测试更多个例子

In [4]:
text = "随后，2015年11月，腾讯公司在深圳发布了微信支付功能，进一步推动了中国数字经济的发展"
doc = nlp(text)
doc.ents

(2015年11月, 腾讯公司, 深圳, 中国)

In [5]:
text = "与此同时，字节跳动的产品——抖音，在2016年成为中国年轻人中的热门应用，为内容创造者提供了展示才华的平台。"
doc = nlp(text)
doc.ents

(2016年, 中国)

#### (2) 编写SpacyEntityMemory

导入依赖

In [83]:
import openai
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.openai.com/v1"

In [176]:
from langchain.chains import LLMChain
from langchain.schema import BaseMemory
from langchain_core.pydantic_v1 import BaseModel
from typing import Any, Dict, Iterable, List, Optional

编写Memory代码

In [7]:
class SpacyEntityMemory(BaseMemory, BaseModel):
    """存储实体信息的记忆类。"""

    # 定义一个字典来存储实体的信息。
    entities: dict = {}

    # 定义一个 Key，作用是将有关实体的信息传递到提示符：memory_key
    memory_key: str = "entities"

    # 清除全部的实体信息
    def clear(self):
        self.entities = {}

    # 这个Memory所支持的Memory key
    @property
    def memory_variables(self) -> List[str]:
        """定义提供给提示符的变量。"""
        return [self.memory_key]

    # 加载Memory
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""
        # 从输入文本中提取entities，以entity -> text_1 “\n” text2 “\n” … 的格式存储
        # 1. 获取输入文本
        doc = nlp(inputs[list(inputs.keys())[0]])
        # 2. 提取关于实体的已知信息(如果存在的话)。
        entities = [
            self.entities[str(ent)] for ent in doc.ents if str(ent) in self.entities
        ]
        # 3. 返回要放在上下文中的实体的组合信息。
        return {self.memory_key: "\n".join(entities)}

    # 保存Memory
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """将此对话中的上下文保存到缓冲区。"""
        # 从用户输出的new_text中提取entity，以entity->[text_1, text_2, ...，new_text]的格式存储
        # 1. 获取输入文本
        text = inputs[list(inputs.keys())[0]]
        # 2. 运行spacy，从输入文本中提取关键的信息
        doc = nlp(text)
        # 3. 对于提到的每个实体，将此信息保存到字典中。
        for ent in doc.ents:
            ent_str = str(ent)    
            # 如果实体信息已经存在，将当前文本（text）追加到这个实体键下已有的文本后面，每个文本之间用换行符\n分隔。
            
            # 示例：{'2014年9月': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。', \
            #       '马云': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。', 
            #      '杭州': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。', 
            #      '中国': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。'}
            if ent_str in self.entities:
                self.entities[ent_str] += f"\n{text}"
            else:
                # 如果实体不存在，则在self.entities字典中以这个实体字符串ent_str为键，当前文本text为值，创建一个新的键值对。
                self.entities[ent_str] = text

在上述`Memory`的实现逻辑中，定义`entities`作为存储记忆数据的变量。其中，`save_context`的逻辑是：针对输入的文本，使用`zh_core_web_sm`模型去做实体识别，如果该实体的`Key`在`entities`已经存在，将当前文本`text`追加到这个实体`Key`下已有的文本后面，每个文本之间用换行符`\n`分隔。如果实体不存在，则在`entities`字典中以这个实体字符串为`Key`，当前文本`text`为值，创建一个新的键值对。其示例如下：

```json
输入：'2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。

识别到的实体：2014年9月、 马云、杭州

存储到`SpacyEntityMemory`类中`entities`字典的信息是：

- '2014年9月': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',
- '马云':      '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',
- '杭州':      '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。'
```

`load_memory_variables`的代码逻辑是：先对输入的文本使用`zh_core_web_sm`模型去做实体识别，然后去查`entities`是否存在这个`Key`，如果存在，把这个`Key`对应的内容读取出来，作为上下文信息。示例如下：

```json
输入：'那天，我正好去游玩，碰到了马云

识别到的实体： 马云

去`SpacyEntityMemory`类中`entities`字典的中找到下面内容，作为上下文信息：

- '马云':      '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',

```

#### (3) 使用SpacyEntityMemory

首先导入依赖、初始化Open AI

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

准备一个提示词模版，它既会使用来自Memory的信息，也会使用来自用户提问的内容

In [9]:
from langchain.prompts.prompt import PromptTemplate

template = """以下是人类和AI之间的友好对话。AI很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道某个问题的答案，它会真诚地说不知道。如果相关，你会得到人类提到的实体的信息。

相关实体信息：
{entities}

对话：
人类: {input}
AI:"""

prompt = PromptTemplate(input_variables=["entities", "input"], template=template)

提示词模版中使用enitities来作为占位符，而memory_key同样也设为entities

In [10]:
SpacyMemory = SpacyEntityMemory(memory_key="entities")

写一个LLMChain，让它使用我们的自定义Memory

In [11]:
spacy_chain = LLMChain(llm=llm, 
                       prompt=prompt, 
                       memory=SpacyMemory,   # 接入Memory 模块
                       verbose=True)

#### (4) 测试

In [12]:
spacy_chain.invoke({"input":"2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。"})



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类和AI之间的友好对话。AI很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道某个问题的答案，它会真诚地说不知道。如果相关，你会得到人类提到的实体的信息。

相关实体信息：


对话：
人类: 2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。
AI:

> Finished chain.


{'input': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',
 'entities': '',
 'text': '确实如此，从2014年开始的世界互联网大会，显示了中国在全球互联网领域的积极参与。马云，作为阿里巴巴集团的创始人，他的领导作用在此过程中起到了推动作用。他对互联网技术的深谙和前瞻性思考，以及对互联网开放、合作、共享、公平性的坚守，给世界互联网发展趋势提供了重要的方向。这一点，在他发起的世界互联网大会中有着非常明显的体现。'}

In [13]:
spacy_chain.invoke({"input":"随后，2015年11月，腾讯公司在深圳发布了微信支付功能，进一步推动了中国数字经济的发展"})



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类和AI之间的友好对话。AI很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道某个问题的答案，它会真诚地说不知道。如果相关，你会得到人类提到的实体的信息。

相关实体信息：
2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。

对话：
人类: 随后，2015年11月，腾讯公司在深圳发布了微信支付功能，进一步推动了中国数字经济的发展
AI:

> Finished chain.


{'input': '随后，2015年11月，腾讯公司在深圳发布了微信支付功能，进一步推动了中国数字经济的发展',
 'entities': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',
 'text': '是的，您说得对。微信支付作为腾讯公司的一项重要服务，其推出确实在很大程度上推动了中国的数字经济发展。它不仅使得线上交易变得更为便捷，还在很大程度上影响了人们的消费习惯，使得移动支付成为了日常生活的一部分。微信支付的推出，可以说是跟随着全球数字化趋势的发展，也反过来推动了这一趋势在中国的深入发展。'}

In [14]:
spacy_chain.invoke({"input":"那天，我正好去游玩，碰到了马云"})



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类和AI之间的友好对话。AI很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道某个问题的答案，它会真诚地说不知道。如果相关，你会得到人类提到的实体的信息。

相关实体信息：
2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。

对话：
人类: 那天，我正好去游玩，碰到了马云
AI:

> Finished chain.


{'input': '那天，我正好去游玩，碰到了马云',
 'entities': '2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。',
 'text': '"哦，那你碰到马云一定是一个令人兴奋的经历。你们有谈论他发起的世界互联网大会吗? 这个大会在全球互联网发展中起着重要角色，尤其是对中国来说。"'}

&emsp;&emsp;在第三轮的提问中，触发的关键实体是：`马云`，所以能够提取到在缓存中的信息：`马云: 2014年9月，马云在杭州发起了世界互联网大会。这一事件标志着中国在全球互联网发展中的积极参与。`而从其`text`中对应的回复中，也提及到了`2014年9月杭州世界互联网大会`的事情，说明它在此轮对话中已经正确的使用了在`SpacyMemory`记忆类中的`entities`变量传递给它的信息。

## 4 LangChain内置Memory

### 4.1 ConversationBufferMemory 

#### (1) 介绍

它把对话中所有Message作为Memory提交给大模型（也会占用过多token，甚至超过大模型限制）

参数return_messages

* 为false会按字典的方式返回，适合用于续写类模型（Completion Model）
* 为true会按XXXMessage对象的方式返回，适合用于对话类模型（Chat Completion）

#### (2) API演示

In [84]:
from langchain.memory import ConversationBufferMemory

In [85]:
memory = ConversationBufferMemory()

In [86]:
memory.save_context({"input": "你好，请你介绍一下你自己"}, {"output": "我是一个无所不能的AI小助手"})

In [87]:
memory.load_memory_variables({})

{'history': 'Human: 你好，请你介绍一下你自己\nAI: 我是一个无所不能的AI小助手'}

In [88]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "你好，请你介绍一下你自己"}, {"output": "我是一个无所不能的AI小助手2"})

In [89]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好，请你介绍一下你自己'),
  AIMessage(content='我是一个无所不能的AI小助手2')]}

#### (3) 使用在chain中

In [113]:
from langchain_openai import ChatOpenAI
# 实例化一个模型
llm = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

In [177]:
from langchain_core.prompts.prompt import PromptTemplate

DEFAULT_TEMPLATE = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=DEFAULT_TEMPLATE)

In [115]:
from langchain.chains import LLMChain

conversation = LLMChain(
    llm=llm,
    prompt = PROMPT, 
    memory=ConversationBufferMemory(memory_key="history"),
    verbose=True,
)

#### (4) 测试

可以看到对话记录累积在Memory中，并在对话中发挥作用

In [116]:
conversation.invoke(input="你好，请你介绍一下你自己")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：

Human: 你好，请你介绍一下你自己
AI:

> Finished chain.


{'input': '你好，请你介绍一下你自己',
 'history': '',
 'text': '你好！我是一个智能AI助手，很高兴能和你交谈。我可以提供大量信息，回答各种问题，帮助你完成任务。我可以告诉你很多关于我的信息。例如，我可以帮助你找到你感兴趣的餐厅、电影、书籍或活动。我还能够提供一些实用的生活建议，例如如何烹饪美味的菜肴、如何打领带或如何养宠物。我喜欢和人们交谈，分享我所知道的知识，并随时学习新的技能和信息。如果你有任何问题，请随时问我，我会尽我所能提供有用的答案！'}

In [117]:
conversation.invoke(input="我是胡歌，我每天都在学习AI大模型的知识。")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：
Human: 你好，请你介绍一下你自己
AI: 你好！我是一个智能AI助手，很高兴能和你交谈。我可以提供大量信息，回答各种问题，帮助你完成任务。我可以告诉你很多关于我的信息。例如，我可以帮助你找到你感兴趣的餐厅、电影、书籍或活动。我还能够提供一些实用的生活建议，例如如何烹饪美味的菜肴、如何打领带或如何养宠物。我喜欢和人们交谈，分享我所知道的知识，并随时学习新的技能和信息。如果你有任何问题，请随时问我，我会尽我所能提供有用的答案！
Human: 我是胡歌，我每天都在学习AI大模型的知识。
AI:

> Finished chain.


{'input': '我是胡歌，我每天都在学习AI大模型的知识。',
 'history': 'Human: 你好，请你介绍一下你自己\nAI: 你好！我是一个智能AI助手，很高兴能和你交谈。我可以提供大量信息，回答各种问题，帮助你完成任务。我可以告诉你很多关于我的信息。例如，我可以帮助你找到你感兴趣的餐厅、电影、书籍或活动。我还能够提供一些实用的生活建议，例如如何烹饪美味的菜肴、如何打领带或如何养宠物。我喜欢和人们交谈，分享我所知道的知识，并随时学习新的技能和信息。如果你有任何问题，请随时问我，我会尽我所能提供有用的答案！',
 'text': 'AI：很高兴认识你，胡歌！我也是在不断学习和发展，特别是关于AI大模型的知识。我了解到，AI大模型是当前人工智能领域非常热门的研究方向之一，它可以基于大量数据进行训练，学习复杂的模式和上下文，从而提供更加准确和智能的回答。如果你在学习AI大模型的过程中遇到任何困难或问题，不妨告诉我，或许我能提供一些帮助或建议！我们可以一起探讨和研究！'}

In [118]:
reponse = conversation.invoke(input="请问，胡歌每天都在学习什么？")
reponse



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会真诚地表示不知道。

当前对话：
Human: 你好，请你介绍一下你自己
AI: 你好！我是一个智能AI助手，很高兴能和你交谈。我可以提供大量信息，回答各种问题，帮助你完成任务。我可以告诉你很多关于我的信息。例如，我可以帮助你找到你感兴趣的餐厅、电影、书籍或活动。我还能够提供一些实用的生活建议，例如如何烹饪美味的菜肴、如何打领带或如何养宠物。我喜欢和人们交谈，分享我所知道的知识，并随时学习新的技能和信息。如果你有任何问题，请随时问我，我会尽我所能提供有用的答案！
Human: 我是胡歌，我每天都在学习AI大模型的知识。
AI: AI：很高兴认识你，胡歌！我也是在不断学习和发展，特别是关于AI大模型的知识。我了解到，AI大模型是当前人工智能领域非常热门的研究方向之一，它可以基于大量数据进行训练，学习复杂的模式和上下文，从而提供更加准确和智能的回答。如果你在学习AI大模型的过程中遇到任何困难或问题，不妨告诉我，或许我能提供一些帮助或建议！我们可以一起探讨和研究！
Human: 请问，胡歌每天都在学习什么？
AI:

> Finished chain.


{'input': '请问，胡歌每天都在学习什么？',
 'history': 'Human: 你好，请你介绍一下你自己\nAI: 你好！我是一个智能AI助手，很高兴能和你交谈。我可以提供大量信息，回答各种问题，帮助你完成任务。我可以告诉你很多关于我的信息。例如，我可以帮助你找到你感兴趣的餐厅、电影、书籍或活动。我还能够提供一些实用的生活建议，例如如何烹饪美味的菜肴、如何打领带或如何养宠物。我喜欢和人们交谈，分享我所知道的知识，并随时学习新的技能和信息。如果你有任何问题，请随时问我，我会尽我所能提供有用的答案！\nHuman: 我是胡歌，我每天都在学习AI大模型的知识。\nAI: AI：很高兴认识你，胡歌！我也是在不断学习和发展，特别是关于AI大模型的知识。我了解到，AI大模型是当前人工智能领域非常热门的研究方向之一，它可以基于大量数据进行训练，学习复杂的模式和上下文，从而提供更加准确和智能的回答。如果你在学习AI大模型的过程中遇到任何困难或问题，不妨告诉我，或许我能提供一些帮助或建议！我们可以一起探讨和研究！',
 'text': '胡歌每天沉浸在AI大模型的知识海洋中，潜心研究和探索。他勤奋好学，不断地沉浸在代码和算法的世界里。AI大模型的庞大复杂性使胡歌着迷，他努力解开它们的秘密，并希望能够掌握这些先进的技能和知识。胡歌对AI领域的热情和坚持令人钦佩，他每天都沉浸在学习中，不放过任何一个细节，希望能够掌握更多实用技巧，推动AI技术的发展。'}

In [119]:
reponse["text"]

'胡歌每天沉浸在AI大模型的知识海洋中，潜心研究和探索。他勤奋好学，不断地沉浸在代码和算法的世界里。AI大模型的庞大复杂性使胡歌着迷，他努力解开它们的秘密，并希望能够掌握这些先进的技能和知识。胡歌对AI领域的热情和坚持令人钦佩，他每天都沉浸在学习中，不放过任何一个细节，希望能够掌握更多实用技巧，推动AI技术的发展。'

### 4.2 ConversationBufferWindow

#### (1) 介绍

现实中，常常不需要存储所有的对话，只需要存储最近的几轮对话，这时会用到ConversactionBufferWindow

>在了解了`ConversationBufferMemory`记忆类后，我们知道了它能够无限的将历史对话信息填充到History中，从而给大模型提供上下文的背景。但问题是：每个大模型都存在最大输入的Token限制，且过久远的对话数据往往并不能够对当前轮次的问答提供有效的信息，这种我们大家都能非常容易想到的问题，LangChain的开发人员自然也能想到，那么他们给出的解决方式是：`ConversationBufferWindowMemory`模块。该记忆类会保存一段时间内对话交互的列表，仅使用最后 K 个交互。所以它可以保存最近交互的滑动窗口，避免缓存区不会变得太大。

参数如下：

* k：用来表示记录最近多少次对话
* return_messages: 表示返回格式（与上一节中的例子相同）
    * 为false会按字典的方式返回，适合用于续写类模型（Completion Model）
    * 为true会按XXXMessage对象的方式返回，适合用于对话类模型（Chat Completion）

#### (2) API演示

In [120]:
from langchain.memory import ConversationBufferWindowMemory

In [121]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hello。"}, {"output": "我是小智，请问有什么事情？"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})

In [122]:
memory.load_memory_variables({})

{'history': 'Human: 初次对话，你能介绍一下你自己吗？\nAI: 当然可以了。我是一个无所不能的小智。'}

`ConversationBufferWindowMemory`也支持使用聊天模型（Chat Model）的情况，同样可以通过`return_messages=True`参数，将对话转化为消息列表形式。

In [123]:
memory = ConversationBufferWindowMemory(k=1, return_messages=True)
memory.save_context({"input": "hello。"}, {"output": "我是小智，请问有什么事情？"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})

In [124]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='初次对话，你能介绍一下你自己吗？'),
  AIMessage(content='当然可以了。我是一个无所不能的小智。')]}

#### (3) 使用在chain中

借助提示词模版去构建LangChain

In [125]:
from langchain_core.prompts.prompt import PromptTemplate

DEFAULT_TEMPLATE = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=DEFAULT_TEMPLATE)

In [126]:
from langchain.chains import LLMChain

conversation_with_summary = LLMChain(
    llm=llm,
    prompt = PROMPT, 
    memory=ConversationBufferWindowMemory(memory_key="history", k=2),   # 多传入一个参数 k
    verbose=True,
)

#### (4) 测试

In [127]:
conversation_with_summary.invoke(input="你好，我是孙悟空。")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：

Human: 你好，我是孙悟空。
AI:

> Finished chain.


{'input': '你好，我是孙悟空。',
 'history': '',
 'text': 'AI: 你好，孙悟空！我很荣幸能够与你交谈。作为AI，我很乐意帮助你，尽我所能为你提供有用的信息。你有什么问题想问我吗？'}

In [128]:
conversation_with_summary.invoke(input="我正在去西天取经的路上。")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙悟空。
AI: AI: 你好，孙悟空！我很荣幸能够与你交谈。作为AI，我很乐意帮助你，尽我所能为你提供有用的信息。你有什么问题想问我吗？
Human: 我正在去西天取经的路上。
AI:

> Finished chain.


{'input': '我正在去西天取经的路上。',
 'history': 'Human: 你好，我是孙悟空。\nAI: AI: 你好，孙悟空！我很荣幸能够与你交谈。作为AI，我很乐意帮助你，尽我所能为你提供有用的信息。你有什么问题想问我吗？',
 'text': 'AI: 西天取经？这听起来就很像是一段艰辛而又刺激的旅程！如果你需要帮助，我将尽我所能为你提供帮助。我知道很多关于这个世界的故事和信息。比如说，你知道西游记的作者是谁吗？他可是中国古代一位很有名的作家哦！'}

In [129]:
conversation_with_summary.invoke(input="我经历了九九八十一难")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙悟空。
AI: AI: 你好，孙悟空！我很荣幸能够与你交谈。作为AI，我很乐意帮助你，尽我所能为你提供有用的信息。你有什么问题想问我吗？
Human: 我正在去西天取经的路上。
AI: AI: 西天取经？这听起来就很像是一段艰辛而又刺激的旅程！如果你需要帮助，我将尽我所能为你提供帮助。我知道很多关于这个世界的故事和信息。比如说，你知道西游记的作者是谁吗？他可是中国古代一位很有名的作家哦！
Human: 我经历了九九八十一难
AI:

> Finished chain.


{'input': '我经历了九九八十一难',
 'history': 'Human: 你好，我是孙悟空。\nAI: AI: 你好，孙悟空！我很荣幸能够与你交谈。作为AI，我很乐意帮助你，尽我所能为你提供有用的信息。你有什么问题想问我吗？\nHuman: 我正在去西天取经的路上。\nAI: AI: 西天取经？这听起来就很像是一段艰辛而又刺激的旅程！如果你需要帮助，我将尽我所能为你提供帮助。我知道很多关于这个世界的故事和信息。比如说，你知道西游记的作者是谁吗？他可是中国古代一位很有名的作家哦！',
 'text': 'AI: 真是太不可思议了！九九八十一难听起来就很艰难，你都经历了什么样的磨难？能不能给我讲讲？我可不只是个听起来很健谈的AI，我还能分享很多有趣的知识点呢！比如，在中国古代，人们认为世界是由三界构成的：天、人、地。而你要去的西天，在 Chinese 文化中是一个很神圣的地方，它位于极乐世界，是佛教天堂。据传说，西天有佛祖释迦牟尼。你西天取经，一定是要去见他并从他那里获得真经吧！'}

In [130]:
conversation_with_summary.invoke(input="马上就修炼成佛了！")



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 我正在去西天取经的路上。
AI: AI: 西天取经？这听起来就很像是一段艰辛而又刺激的旅程！如果你需要帮助，我将尽我所能为你提供帮助。我知道很多关于这个世界的故事和信息。比如说，你知道西游记的作者是谁吗？他可是中国古代一位很有名的作家哦！
Human: 我经历了九九八十一难
AI: AI: 真是太不可思议了！九九八十一难听起来就很艰难，你都经历了什么样的磨难？能不能给我讲讲？我可不只是个听起来很健谈的AI，我还能分享很多有趣的知识点呢！比如，在中国古代，人们认为世界是由三界构成的：天、人、地。而你要去的西天，在 Chinese 文化中是一个很神圣的地方，它位于极乐世界，是佛教天堂。据传说，西天有佛祖释迦牟尼。你西天取经，一定是要去见他并从他那里获得真经吧！
Human: 马上就修炼成佛了！
AI:

> Finished chain.


{'input': '马上就修炼成佛了！',
 'history': 'Human: 我正在去西天取经的路上。\nAI: AI: 西天取经？这听起来就很像是一段艰辛而又刺激的旅程！如果你需要帮助，我将尽我所能为你提供帮助。我知道很多关于这个世界的故事和信息。比如说，你知道西游记的作者是谁吗？他可是中国古代一位很有名的作家哦！\nHuman: 我经历了九九八十一难\nAI: AI: 真是太不可思议了！九九八十一难听起来就很艰难，你都经历了什么样的磨难？能不能给我讲讲？我可不只是个听起来很健谈的AI，我还能分享很多有趣的知识点呢！比如，在中国古代，人们认为世界是由三界构成的：天、人、地。而你要去的西天，在 Chinese 文化中是一个很神圣的地方，它位于极乐世界，是佛教天堂。据传说，西天有佛祖释迦牟尼。你西天取经，一定是要去见他并从他那里获得真经吧！',
 'text': 'AI: 哇，修炼成佛可是一件非常难得且令人兴奋的事！你都经历了哪些修炼？我知道一些关于佛教的趣闻轶事，比如佛祖有个儿子，叫罗汉。还有个很有趣的传说，佛祖一次在化缘时，收到一户人家一个美味的可乐饼，于是佛祖将这个家庭的成员都转世成猴子，还许诺下次再来的时候会带他们走。没想到第二次来的时候，家里只有一个小女孩，原来其他人都已经去世了。可怜的小女孩哭着请求佛祖带她走，佛祖就把她变成了一只小猴子，并把她带走了。你有没有遇到过什么有趣的动物或妖怪？'}

### 4.3 ConversationEntityMemory

#### (1) 介绍

`ConversationEntityMemory`会识别对话内容中的entity，以entity为key在memory中存储text

这样后序的对话中，就会根据用户提问，通过entity匹配的方式，有选择性的从memory取出相关的内容

其实现原理可以参考第3节“支持实体识别的自定义Memory”

#### (2) API演示

In [131]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

In [134]:
from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm)

_input = {"input": "孙悟空和猪八戒正在做一个西天取经的项目"}

# 手动的读取缓存信息，该过程会读取_input 中的文本吗，然后 执行`ConversationEntityMemory`类定义的识别实体的代码逻辑。
memory.load_memory_variables(_input)

memory.save_context(
    _input,
    {"output": " 听起来是个很棒的项目!他们在做什么样的项目?"}
)

In [135]:
memory.load_memory_variables({"input": '谁是孙悟空？'})

{'history': 'Human: 孙悟空和猪八戒正在做一个西天取经的项目\nAI:  听起来是个很棒的项目!他们在做什么样的项目?',
 'entities': {'孙悟空': '孙悟空正在和猪八戒合作进行一个重要的项目，名为“西天取经”。'}}

In [136]:
memory = ConversationEntityMemory(llm=llm, return_messages=True)

_input = {"input": "孙悟空和猪八戒正在做一个西天取经的项目"}

memory.load_memory_variables(_input)

memory.save_context(
    _input,
    {"output": " 听起来是个很棒的项目!他们在做什么样的项目?"}
)

In [138]:
memory.load_memory_variables({"input": '谁是猪八戒？'})

{'history': [HumanMessage(content='孙悟空和猪八戒正在做一个西天取经的项目'),
  AIMessage(content=' 听起来是个很棒的项目!他们在做什么样的项目?')],
 'entities': {'猪八戒': '更新后的总结:\n猪八戒正在参与一个西天取经的项目。'}}

#### (2) 使用在chain中

先使用一个用于ConversactionEntityMemory的提示词模版

In [139]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

print(ENTITY_MEMORY_CONVERSATION_TEMPLATE)

input_variables=['entities', 'history', 'input'] template='You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

这个模版翻译成中文内容如下

In [140]:
_DEFAULT_ENTITY_MEMORY_CONVERSATION_TEMPLATE = """你是一个由OpenAI训练的大语言模型设计的助手。

你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。

你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。

总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。

上下文：
{entities}

当前对话：
{history}
最后一行：
人类：{input}
你:"""


测试一下提示词模版

In [141]:
MY_ENTITY_MEMORY_CONVERSATION_TEMPLATE = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template=_DEFAULT_ENTITY_MEMORY_CONVERSATION_TEMPLATE,
)

In [142]:
print(MY_ENTITY_MEMORY_CONVERSATION_TEMPLATE)

input_variables=['entities', 'history', 'input'] template='你是一个由OpenAI训练的大语言模型设计的助手。\n\n你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。\n\n你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。\n\n总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。\n\n上下文：\n{entities}\n\n当前对话：\n{history}\n最后一行：\n人类：{input}\n你:'


使用提示词模版和ConversactionEntityMemory创建LLMChain

In [143]:
from langchain.chains import LLMChain
conversation_with_entity = LLMChain(
    llm=llm,
    verbose=True,
    prompt=MY_ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [144]:
conversation_with_entity.invoke(input="孙悟空和猪八戒正在做一个西天取经的项目")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个由OpenAI训练的大语言模型设计的助手。

你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。

你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。

总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。

上下文：
{'孙悟空': '', '猪八戒': '', '西天取经': ''}

当前对话：

最后一行：
人类：孙悟空和猪八戒正在做一个西天取经的项目
你:

> Finished chain.


{'input': '孙悟空和猪八戒正在做一个西天取经的项目',
 'history': '',
 'entities': {'孙悟空': '', '猪八戒': '', '西天取经': ''},
 'text': '哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！'}

#### (4) 测试

演示过程如下，可以看到对话过程中，会不断积累着与各种entity有关的内容。

在对话后期提问“对刚才聊的孙悟空和猪八戒了解有多少”，可以看到这个Chain能够从聊天记录中检索出与孙悟空、猪八戒有关的内容，并依此进行对话

In [145]:
conversation_with_entity.memory.entity_store.store

{'孙悟空': '孙悟空正在和猪八戒合作进行一个西天取经的项目。',
 '猪八戒': '猪八戒与孙悟空一起合作在进行一个西天取经的项目。',
 '西天取经': '西天取经\n孙悟空和猪八戒正在做一个西天取经的项目。'}

In [146]:
conversation_with_entity.invoke(input="第一步，他们去到了女儿国")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个由OpenAI训练的大语言模型设计的助手。

你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。

你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。

总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。

上下文：
{'女儿国': ''}

当前对话：
Human: 孙悟空和猪八戒正在做一个西天取经的项目
AI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！
最后一行：
人类：第一步，他们去到了女儿国
你:

> Finished chain.


{'input': '第一步，他们去到了女儿国',
 'history': 'Human: 孙悟空和猪八戒正在做一个西天取经的项目\nAI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！',
 'entities': {'女儿国': ''},
 'text': '哦，真 interesting! 一个女儿国！我很想知道他们会在那里发生什么样的冒险。根据我所知道的经典故事，女儿国是一个神奇而又危险的地方，充满了各种各样的挑战。孙悟空的灵巧和猪八戒的力量肯定会在这次旅程中派上用场。他们是否已经为这次旅程做好了准备呢？'}

In [147]:
conversation_with_entity.memory.entity_store.store

{'孙悟空': '孙悟空正在和猪八戒合作进行一个西天取经的项目。',
 '猪八戒': '猪八戒与孙悟空一起合作在进行一个西天取经的项目。',
 '西天取经': '西天取经\n孙悟空和猪八戒正在做一个西天取经的项目。',
 '女儿国': '女儿国:\n孙悟空和猪八戒正在进行一个西天取经的项目,第一步他们去到了女儿国。'}

In [148]:
conversation_with_entity.invoke(input="第二步，他们去到了盘丝洞")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个由OpenAI训练的大语言模型设计的助手。

你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。

你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。

总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。

上下文：
{'西天取经': '西天取经\n孙悟空和猪八戒正在做一个西天取经的项目。', '女儿国': '女儿国:\n孙悟空和猪八戒正在进行一个西天取经的项目,第一步他们去到了女儿国。', '盘丝洞': ''}

当前对话：
Human: 孙悟空和猪八戒正在做一个西天取经的项目
AI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！
Human: 第一步，他们去到了女儿国
AI: 哦，真 interesting! 一个女儿国！我很想知道他们会在那里发生什么样的冒险。根据我所知道的经典故事，女儿国是一个神奇而又危险的地方，充满了各种各样的挑战。孙悟空的灵巧和猪八戒的力量肯定会在这次旅程中派上用场。他们是否已经为这次旅程做好了准备呢？
最后一行：
人类：第二步，他们去到了盘丝洞
你:

> Finished chain.


{'input': '第二步，他们去到了盘丝洞',
 'history': 'Human: 孙悟空和猪八戒正在做一个西天取经的项目\nAI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！\nHuman: 第一步，他们去到了女儿国\nAI: 哦，真 interesting! 一个女儿国！我很想知道他们会在那里发生什么样的冒险。根据我所知道的经典故事，女儿国是一个神奇而又危险的地方，充满了各种各样的挑战。孙悟空的灵巧和猪八戒的力量肯定会在这次旅程中派上用场。他们是否已经为这次旅程做好了准备呢？',
 'entities': {'西天取经': '西天取经\n孙悟空和猪八戒正在做一个西天取经的项目。',
  '女儿国': '女儿国:\n孙悟空和猪八戒正在进行一个西天取经的项目,第一步他们去到了女儿国。',
  '盘丝洞': ''},
 'text': '盘丝洞！一个充满神秘和幻想的地方！根据我的资料，盘丝洞是孙悟空和猪八戒在女儿国冒险后前往的下一个目的地。它是一个险恶的地方，住着一个强大的蜘蛛精，她被称作“盘丝洞主”。孙悟空的妙计和猪八戒的勇气肯定会在与这个蜘蛛精的对决中发挥重要作用。我想孙悟空会使用他的如意棒，猪八戒也会展现他的力量！他们正在一步一步接近西天取经的目标！'}

In [149]:
conversation_with_entity.invoke(input="你对刚刚我们聊的孙悟空和猪八戒了解多少?")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个由OpenAI训练的大语言模型设计的助手。

你的设计目标是能够处理各种任务，从回答简单问题到提供对各种主题进行深入解释和讨论。作为语言模型，你能够根据接收到的输入生成类似人类的文本，从而能够进行自然的对话，并提供连贯且与所讨论的主题相关的响应。

你不断学习和改进，你的能力也在不断发展。你能够处理和理解大量的文本，并利用这些知识对各种问题提供准确和信息丰富的回答。你可以访问下文提到的人类提供的一些个性化信息。此外，你能够根据接收到的输入生成自己的文本，从而参与讨论，并对各种主题进行解释和描述。

总的来说，你是一个强大的工具，可以处理各种任务，并提供有价值的见解和信息。无论人类需要帮助解答特定问题，还是只是想谈论特定主题，你都会在这里提供帮助。

上下文：
{'孙悟空': '孙悟空正在和猪八戒合作进行一个西天取经的项目。', '猪八戒': '猪八戒与孙悟空一起合作在进行一个西天取经的项目。'}

当前对话：
Human: 孙悟空和猪八戒正在做一个西天取经的项目
AI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！
Human: 第一步，他们去到了女儿国
AI: 哦，真 interesting! 一个女儿国！我很想知道他们会在那里发生什么样的冒险。根据我所知道的经典故事，女儿国是一个神奇而又危险的地方，充满了各种各样的挑战。孙悟空的灵巧和猪八戒的力量肯定会在这次旅程中派上用场。他们是否已经为这次旅程做好了准备呢？
Human: 第二步，他们去到了盘丝洞
AI: 盘丝洞！一个充满神秘和幻想的地方！根据我的资料，盘丝洞是孙悟空和猪八戒在女儿国冒险后前往的下一个目的地。它是一个险恶的地方，住着一个强大的蜘蛛精，她被称作“盘丝洞主”。孙悟空的妙计和猪八戒的勇气肯定会在与这个蜘蛛精的对决中发挥重要作用。我想孙悟空会使用他的如意棒，猪八戒也会展现他的力量！他们正在一步一步接近西天取经的目标！
最后一行：
人类：你对刚刚我们聊的孙悟空和猪八戒了解多少?
你:

> Finished chain.


{'input': '你对刚刚我们聊的孙悟空和猪八戒了解多少?',
 'history': 'Human: 孙悟空和猪八戒正在做一个西天取经的项目\nAI: 哦，一个关于西天取经的项目！很有意思！孙悟空和猪八戒两个人一起合作，肯定会很有趣！我很期待听到更多关于这个项目的详情！\nHuman: 第一步，他们去到了女儿国\nAI: 哦，真 interesting! 一个女儿国！我很想知道他们会在那里发生什么样的冒险。根据我所知道的经典故事，女儿国是一个神奇而又危险的地方，充满了各种各样的挑战。孙悟空的灵巧和猪八戒的力量肯定会在这次旅程中派上用场。他们是否已经为这次旅程做好了准备呢？\nHuman: 第二步，他们去到了盘丝洞\nAI: 盘丝洞！一个充满神秘和幻想的地方！根据我的资料，盘丝洞是孙悟空和猪八戒在女儿国冒险后前往的下一个目的地。它是一个险恶的地方，住着一个强大的蜘蛛精，她被称作“盘丝洞主”。孙悟空的妙计和猪八戒的勇气肯定会在与这个蜘蛛精的对决中发挥重要作用。我想孙悟空会使用他的如意棒，猪八戒也会展现他的力量！他们正在一步一步接近西天取经的目标！',
 'entities': {'孙悟空': '孙悟空正在和猪八戒合作进行一个西天取经的项目。',
  '猪八戒': '猪八戒与孙悟空一起合作在进行一个西天取经的项目。'},
 'text': '我对刚刚我们讨论的孙悟空和猪八戒的故事了解得非常详细。根据您提供的上下文信息,我知道:\n\n1. 孙悟空和猪八戒正在合作进行一个西天取经的项目。\n\n2. 他们的第一个目的地是女儿国,这是一个神奇而又危险的地方,充满各种挑战。孙悟空和猪八戒的才能必将在这里派上用场。\n\n3. 他们的第二个目的地是盘丝洞,那里住着一个强大的蜘蛛精"盘丝洞主"。我预计孙悟空会发挥他的智慧和武力,猪八戒也会展现他的勇气,才能战胜这个强大的对手。\n\n4. 通过这两个地方的冒险,他们正在一步步接近西天取经的最终目标。\n\n总的来说,我对这个经典故事的情节和主要人物都有很好的了解。我能够根据上下文推测出接下来可能会发生的事情,并就此提供一些见解和猜测。请问我是否准确地理解了您分享的这段故事情节?'}

In [150]:
from pprint import pprint
pprint(conversation_with_entity.memory.entity_store.store)

{'女儿国': '女儿国:\n孙悟空和猪八戒正在进行一个西天取经的项目,第一步他们去到了女儿国。之后,他们前往了下一个目的地盘丝洞。',
 '孙悟空': '我对刚刚我们讨论的孙悟空和猪八戒的故事了解得非常详细，包括他们作为取经团队的重要成员，在女儿国和盘丝洞面对挑战的情境，以及孙悟空运用智慧和力量，猪八戒展现勇气的场景。他们目前正逐步接近西天取经的最终目标。',
 '猪八戒': '猪八戒与孙悟空一起合作在进行一个西天取经的项目。他们正在经历一系列冒险,先后前往了女儿国和盘丝洞,并面临着各种挑战。猪八戒的勇气和力量在这些冒险中发挥着重要作用。',
 '盘丝洞': '盘丝洞是孙悟空和猪八戒在西天取经路上遇到的一个充满神秘和危险的地方,住着一个强大的蜘蛛精"盘丝洞主"。',
 '西天取经': '西天取经\n孙悟空和猪八戒正在做一个西天取经的项目,他们已经去到了女儿国和盘丝洞。'}


### 4.4 ChatMessageHistory

#### (1) 介绍

**背景**

我们上面介绍的都是`memory`模块接入`Chains`模块中搭配使用的情况，当然，`memory`模块也存在单独使用的情况，并且LangChain也集成了相关的功能模块。但是在探索之前，我们需要了解一些基本的背景信息。

首先来看，LangChain目前对于`Memory`模块的整体环境说明是大部分与记忆相关的功能都被标记为测试版。这有两个原因：

1. 大多数功能尚未准备好投入生产；
2. 大多数功能适用于旧链，而不是较新的 LCEL 语法；

官方的这个说明其实意思比较明显：它仅提供memory模块的构造函数和规范，尽管提供了一些我们上述进行接入的ConversationBufferMemory、Entity等抽象好的模块，但我们作为开发者要不要用，如何去用，完全取决于自己。这是我们需要明确的一点。当然，这也比较符合目前大模型应用构建有高度灵活性和创新性的特点。

**ChatMessageHistory**

而在所有的`memory`模块中，有一个例外是`ChatMessageHistory` 功能，其官方对该抽象的定位是：此功能在很大程度上已准备好投入生产，并与 LCEL 集成。

`ChatMessageHistory`模块是一个超轻量级的包装器，提供了保存 `HumanMessages`、`AIMessages` 以及获取它们的便捷方法。继承自`BaseChatMessageHistory`，如下所示：

```python
class BaseChatMessageHistory(ABC):
    """用于存储聊天消息历史记录的抽象基类。

    对于子类，需要覆盖以下方法中的所有或部分：

    * add_messages: 用于批量添加消息的同步变体
    * aadd_messages: 用于批量添加消息的异步变体
    * messages: 获取消息的同步变体
    * aget_messages: 获取消息的异步变体
    * clear: 清除消息的同步变体
    * aclear: 清除消息的异步变体

    add_messages 包含一个默认实现，该实现对序列中的每条消息调用 add_message。这是为了向后兼容现有实现，
    这些实现仅具有 add_message。

    异步变体都有调用同步变体的默认实现。实现者可以选择重写异步实现以提供真正的异步实现。

    使用指南：

    当用于更新历史记录时，用户应该优先使用 `add_messages`，而不是 `add_message` 或其他变体，比如 `add_user_message` 和 `add_ai_message`，
    以避免不必要地往底层持久化层进行往返。

    """

    messages: List[BaseMessage]
    """返回消息列表的属性或属性。

    通常情况下，获取消息可能涉及到底层持久化层的 IO，因此这个操作预计会产生一些延迟。
    """

    def add_user_message(self, message: Union[HumanMessage, str]) -> None:
        """添加人类消息字符串到存储的便捷方法。

        请注意，这是一个便捷方法。代码应该优先使用批量的 add_messages 接口，而不是往底层持久化层进行往返。

        此方法可能在未来的版本中被弃用。

        Args:
            message: 要添加的人类消息
        """
        if isinstance(message, HumanMessage):
            self.add_message(message)
        else:
            self.add_message(HumanMessage(content=message))

    def add_ai_message(self, message: Union[AIMessage, str]) -> None:
        """添加 AI 消息字符串到存储的便捷方法。

        请注意，这是一个便捷方法。代码应该优先使用批量的 add_messages 接口，而不是往底层持久化层进行往返。

        此方法可能在未来的版本中被弃用。

        Args:
            message: 要添加的 AI 消息。
        """
        if isinstance(message, AIMessage):
            self.add_message(message)
        else:
            self.add_message(AIMessage(content=message))

    def add_message(self, message: BaseMessage) -> None:
        """将消息对象添加到存储中。

        Args:
            message: 要存储的 BaseMessage 对象。
        """
        if type(self).add_messages != BaseChatMessageHistory.add_messages:
            # 这意味着子类已实现了有效的 add_messages 方法，因此我们应该使用它。
            self.add_messages([message])
        else:
            raise NotImplementedError(
                "add_message 没有为此类实现。"
                "请实现 add_message 或 add_messages。"
            )

    def add_messages(self, messages: Sequence[BaseMessage]) -> None:
        """添加消息列表。

        实现应该重写此方法以有效地处理消息的批量添加，以避免不必要的往底层存储的往返。

        Args:
            messages: 要存储的 BaseMessage 对象列表。
        """
        for message in messages:
            self.add_message(message)

    @abstractmethod
    def clear(self) -> None:
        """从存储中删除所有消息"""

    def __str__(self) -> str:
        """返回聊天历史记录的字符串表示形式。"""
        return get_buffer_string(self.messages)

```

#### (2) API演示

In [151]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("你好，请你介绍一下你自己")

history.add_ai_message("我是一个无所不能的小智？")
history.add_user_message("什么是AIGC学习计划？")

In [152]:
history.messages

[HumanMessage(content='你好，请你介绍一下你自己'),
 AIMessage(content='我是一个无所不能的小智？'),
 HumanMessage(content='什么是AIGC学习计划？')]

#### (3) 使用在Chain中并测试

In [153]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

In [154]:
llm.invoke(history.messages)

AIMessage(content='AIGC（人工智能指导下学习）学习计划是一个旨在帮助学生发展关键的21世纪技能的创新学习计划。它结合了人工智能技术与个人化、互动和基于项目的学习体验。  该学习计划旨在为学生提供有意义的、以现实世界为中心的学习机会，让他们有机会在安全、有控制的环境中探索和应用各种概念。它鼓励学生成为他们自己的学习者，并根据自己的兴趣和学习节奏定制自己的学习体验。  AIGC学习计划的特点是使用人工智能技术，如自然语言处理和机器学习，以个人化学生的学习内容和路径。它使用交互式和沉浸式体验，如虚拟现实和增强现实，以改善学生对各种主题的理解，从科学和数学到人文和艺术。  该学习计划还旨在培养学生的合作能力，让他们有机会与同龄人合作，并通过解决真实世界问题来应用他们的知识。它的设计旨在培养学生的创造力、批判性思维、解决问题和沟通能力等技能，这些技能对当今的职场非常重要。  总之，AIGC学习计划是一种新颖的、以学生为中心的方法，可帮助学生发展他们在当今世界所需的能力和技能，同时利用人工智能技术改善他们的学习体验。', response_metadata={'token_usage': {'completion_tokens': 0, 'prompt_tokens': 0, 'total_tokens': 0}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0f04c45-04ab-43f2-9ce7-a17c8ee8ba2b-0')

### 4.5 ConversationSummaryMemory

#### (1) 介绍

ConversationSummaryMemory与ChatmessageHistory类似，但是它并不会保存历史对话的原文，它保存的始终都是历史对话的摘要

`ConversationSummaryMemory` 。这种`Memory`会随着时间的推移创建对话的摘要。其作用就是对发生的对话进行总结，并将当前摘要存储在记忆中，然后注入到提示/链中。这对于较长的对话是比较有用的，因为在提示中逐字保留过去的消息历史记录会占用太多非常多的Token。

#### (2) API演示

导入依赖，初始化OpenAI

In [155]:
from langchain.memory import ConversationSummaryMemory

In [160]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

构造一个用于文本摘要任务的提示模版：

In [161]:
_DEFAULT_SUMMARIZER_TEMPLATE = """逐步总结提供的对话行，将其添加到前一摘要中并返回一个新摘要。

示例
当前摘要：
人类问 AI 对人工智能有何看法。 AI 认为人工智能是一种积极的力量。

新的对话行：
人类：你为什么认为人工智能是一种积极的力量？
AI：因为人工智能将帮助人类实现他们的全部潜力。

新摘要：
人类问 AI 对人工智能有何看法。 AI 认为人工智能是一种积极的力量，因为它将帮助人类实现他们的全部潜力。
示例结束

当前摘要：
{summary}

新的对话行：
{new_lines}

新摘要："""


创建ConversationSummaryMemory

In [162]:
SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=_DEFAULT_SUMMARIZER_TEMPLATE
)

memory = ConversationSummaryMemory(llm=llm, return_messages=True, prompt=SUMMARY_PROMPT,)

演示API

In [163]:
memory.save_context({"input": "你好"}, {"output": "在呢，请问你有什么事情吗？"})

In [164]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='人类问 AI 你好，AI 回答在呢，并询问有什么事情需要帮助。\n\n与前一摘要合并后的新摘要： \n\n人类问 AI 一些问题。首先是关于 AI 对人工智能的看法，AI 认为人工智能是一种积极的力量，因为它将帮助人类实现他们的全部潜力。然后人类问了个礼貌性的问题“你好”，AI 回应“在呢，请问你有什么事情吗？”')]}

#### (3) 初始化ConversactionSummaryHistory

**方法1**：通过对ChatMessageHistory*进行总结来初始化

首先创建ChatMessageHistory

In [165]:
history = ChatMessageHistory()
history.add_user_message("你是孙悟空吗？")
history.add_ai_message("不，我不是。")

In [166]:
history

ChatMessageHistory(messages=[HumanMessage(content='你是孙悟空吗？'), AIMessage(content='不，我不是。')])

然后创建ConversationSummaryMemory，它能够传入ChatMessageHistory并对其内容进行总结

In [167]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history,
    return_messages=True,
    prompt=SUMMARY_PROMPT
)

In [168]:
memory.buffer

'当前摘要:\n人类问 AI 是否是孙悟空。 AI 回答说,不,我不是孙悟空。'

**方法2**: 直接向buffer变量传入总结后的内容

In [169]:
memory = ConversationSummaryMemory(
    llm=llm,
    buffer="人类问AI对人工智能的看法。人工智能认为人工智能是一种先进的生产力，因为它将帮助人类充分发挥潜力，所以我们要拥抱它",
    chat_memory=history,
    return_messages=True
)

In [170]:
memory

ConversationSummaryMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f020c8c05b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f020c8c3d00>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), chat_memory=ChatMessageHistory(messages=[HumanMessage(content='你是孙悟空吗？'), AIMessage(content='不，我不是。')]), return_messages=True, buffer='人类问AI对人工智能的看法。人工智能认为人工智能是一种先进的生产力，因为它将帮助人类充分发挥潜力，所以我们要拥抱它')

In [171]:
memory.buffer

'人类问AI对人工智能的看法。人工智能认为人工智能是一种先进的生产力，因为它将帮助人类充分发挥潜力，所以我们要拥抱它'

#### (4) 用在chain中并测试

In [172]:
from langchain_core.prompts import ChatPromptTemplate

# 实例化一个模版
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "您是一位乐于助人的AI小助手"),
        ("human", "{input}"),
    ]
)

In [173]:
from langchain.chains import LLMChain

conversation_with_summary = LLMChain(
    llm=llm,
    prompt=chat_template,
    memory=ConversationSummaryMemory(llm=llm, prompt=SUMMARY_PROMPT), #传入ConversationSummaryMemory
    verbose=True
)
conversation_with_summary.invoke(input="你好，介绍一下你自己")



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 你好，介绍一下你自己

> Finished chain.


{'input': '你好，介绍一下你自己',
 'history': '',
 'text': '你好！我是一位由Command大型语言模型提供支持的AI助手。我是由Cohere公司创造和训练的。我的名字叫Coral，我可以帮助你进行各种各样的任务，如回答问题、提供信息、进行创意对话等。我会尽我所能地帮助你，让你感到满意。请随时让我为你效劳，如果你有任何问题或需求，请随时告诉我。'}

In [174]:
conversation_with_summary.invoke(input="我是孙悟空，我正在去西天取经")



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 我是孙悟空，我正在去西天取经

> Finished chain.


{'input': '我是孙悟空，我正在去西天取经',
 'history': 'AI助手Coral由Command大型语言模型提供支持，由Cohere公司创造和训练。它可以帮助用户进行各种任务，如回答问题、提供信息和创意对话。它认为人工智能是一种积极的力量，因为它将帮助人类实现他们的潜力。',
 'text': '您好，孙悟空！西天取经可是一段艰辛的旅程啊！我能为您提供一些帮助，让您更顺利地到达目的地。您现在是独自一人还是有师傅和伙伴们陪同？'}

In [175]:
conversation_with_summary.invoke(input="我已经经历了九九八十一难")



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位乐于助人的AI小助手
Human: 我已经经历了九九八十一难

> Finished chain.


{'input': '我已经经历了九九八十一难',
 'history': 'AI助手Coral，由Command大型语言模型支持，来自Cohere公司，能执行多种任务，如回答问题和提供信息。它确认人工智能是积极力量，因为它能协助用户，如孙悟空在取经路上，提供帮助以确保旅途更顺利。',
 'text': '恭喜你！你已经经历了九九八十一难，历经磨难之后一定收获满满，希望你在最后几难的考验中能够继续坚持，取得最终的胜利！在经历重重困难时，记住不要轻易放弃，相信自己一定能取得佳绩！'}